# Parametric types and arrays

Arrays are a good example to get some idea of the Julian way of defining interfaces.

## Parametric types

Defining arrays is rather intuitive, for example:

In [ ]:
i = [1, 2, 3, 4]    # An integer array

In [ ]:
f = [1.2, 3.4, 21, π]  # A float array ... note the implicit conversion

In [ ]:
s = ["abc", "def", "ghi"]  # A string array

Notice, that the type of arrays is `Array{T, N}`, where `T` is a type and `N` is the number of dimensions. This type is an example of a **parametric type**, i.e. a type, which itself is parametrised by other values or types. For convenience:

In [ ]:
Vector{String} === Array{String, 1}

Arrays do not need to be of one type only, for example ...

In [ ]:
m = [1, 3.4, "abc"]   # A mixed array

and we can explicitly fix their types

In [ ]:
m = Float32[1, 3.4, 4]

Comprehensions also work in Julia:


In [ ]:
arr = randn(10)  # Array of 10 random values
[e for e in arr if e > 0]

Another example of a parametric type is the `Tuple`.

In [ ]:
(1, 2.0, "3")

In [ ]:
typeof((1, 2.0, "3"))

### Type parameters in function signatures

Since `Vector{Real} === Array{Real,1}` one would naively define

In [ ]:
myfunc(v::Vector{Real}) = "Got a real vector!"

However ...

In [ ]:
myfunc([1.0, 2.0, 3.0])

Why is this?

Note although we have

In [ ]:
Float64 <: Real

parametric types have the following (perhaps somewhat counterintuitive) property

In [ ]:
Vector{Float64} <: Vector{Real}

In [ ]:
[1.0, 2.0, 3.0] isa Vector{Real}

How can we understand the behavior above? The crucial point is that `Vector{Real}` is a **concrete** container type despite the fact that `Real` is an abstract type. Specifically, it describes a **heterogeneous** vector of values that individually can be of any type `T <: Real`.

In [ ]:
isconcretetype(Vector{Real})

In [ ]:
[1.0, 2.0]

In [ ]:
Real[1, 2.2, 13f0]

As we have learned above, concrete types are the leafes of the type tree and **cannot** have any subtypes. Hence it is only consistent to have...

In [ ]:
Vector{Float64} <: Vector{Real}

What we often actually *mean* when writing `myfunc(v::Vector{Real}) = ...` is

In [ ]:
myfunc(v::Vector{T}) where T<:Real = "I'm a real vector!"

In [ ]:
myfunc([1.0, 2.0, 3.0])

It works! But what does it mean exactly? First of all, we see that

In [ ]:
Vector{Float64} <: Vector{T} where T<:Real

Here, `Vector{T} where T <: Real` describes the **set** of concrete `Vector` types whose elements are of any specific single type `T` that is a subtype of `Real`.

Think of it as representing `{{ Vector{Float64}, Vector{Int64}, Vector{Int32}, Vector{AbstractFloat}, ... }}`.

In [ ]:
Vector{Int64} <: Vector{T} where T<:Real

In [ ]:
Vector{AbstractFloat} <: Vector{T} where T<:Real

In [ ]:
[1.0, 2.0, 3.0] isa Vector{T} where T<:Real

We can also use the `where` notation to write out our naive `Vector{Real}` from above in a more explicit way:

In [ ]:
Vector{Real} === Vector{T where T<:Real}

Note that the crucial difference is the position of the `where T<:Real` piece, i.e. whether it is inside or outside of the curly braces.

In [ ]:
Vector{T where T<:Real} <: Vector{T} where T<:Real

In [ ]:
(Vector{T} where T<:Real) <: Vector{T where T<:Real}

## Basic functions for arrays

In [ ]:
A = randn(Float32, 4, 5)

In [ ]:
ndims(A)    # Get the number of dimensions

In [ ]:
eltype(A)   # Get the type of the array elements

In [ ]:
length(A)   # Return the number of elements

In [ ]:
size(A)     # Get the size of the array

In [ ]:
size(A, 1)  # Get the size along an axis

In [ ]:
reshape(A, 2, 5, 2)   # Return an array with the shape changed

In [ ]:
A[:, 1]     # Access first column

In [ ]:
A[2, :]     # Access second row

Julia provides the `push!` and `append!` functions to add additional elements to an existing array.
For example:

In [ ]:
A = Vector{Float64}()  # Create an empty Float64 array

In [ ]:
push!(A, 4.)

In [ ]:
append!(A, [5, 6, 7])

Notice, that the `!` is part of the name of the function. In Julia the `!` is a convention to indicate that the respective function *mutates* the content of at least one of the passed arrays.

Very helpful functions as we will see are:

- `zero`, which allocates an array of zeros of the same element type

In [ ]:
A = randn(Float32, 3, 4)
zero(A)

- `similar`, which returns an uninitialised array, which is similar to the passed array. This means that by default array type, element type and size are all kept.

In [ ]:
similar(A)

- One may also change these parameters easily:

In [ ]:
similar(A, (3, 2))            # Keep element type and array type

In [ ]:
similar(A, Float64)           # Change element type

In [ ]:
similar(A, Float64, (1, 2))   # Change element type and shape

## Vector operations and vectorised operations

Array addition (`+`, `-`) and scalar multiplication are directly available on arrays (of any dimension):

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]

In [ ]:
x + 2.0y

For element-wise operations the vectorisation syntax is used:

In [ ]:
x .* y  # elementwise mulitplication

In [ ]:
x .^ y  # Elementwise exponentiation

Note, that the `.`-syntax continues to *all* functions in Julia. That includes base Julia ...

In [ ]:
sqrt.(cos.(2π .* x) .+ sin.(2π * x))

In [ ]:
@. sqrt(cos(2π * x) + sin(2π * x))

... custom functions ...

In [ ]:
myfun(x) = x * x + x
myfun.(y)

... and may be easily chained 

In [ ]:
@. exp(cos(x^2))

### Exercise
Create the following arrays using Julia code:
$$\left(\begin{array}{ccccc}
   2&2&2&2&2 \\
   2&2&2&2&2 \\
   2&2&2&2&2 \\
   \end{array}\right) \qquad
   \left(\begin{array}{cccc}
   0.1&0.5&0.9&1.3\\
   0.2&0.6&1.0&1.4\\
   0.3&0.7&1.1&1.5\\
   0.4&0.8&1.2&1.6\\
   \end{array}\right)
   $$

##### More details on Arrays
- https://docs.julialang.org/en/v1/manual/arrays/
- https://docs.julialang.org/en/v1/base/arrays/
- https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1

## `UnitRange` as an `AbstractArray`

In [ ]:
x = 1:30

In [ ]:
typeof(x)

In [ ]:
typeof(x) <: AbstractArray

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [ ]:
x[3]

In [ ]:
size(x)

In [ ]:
eltype(x)

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [ ]:
fieldnames(typeof(x))

or just by inspecting the source code

In [ ]:
@which UnitRange(1, 30)

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Julia is pretty smart here, for example:

In [ ]:
(1:10) .+ 3

Allocating memory is typically costly.

In [ ]:
collect(1:10)

In [ ]:
@time collect(1:10000000);

But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [ ]:
@time 1:10000000;

Yet, in code they *act* the same way.

## Takeaways

- Parametric types ar types that by themselves have parameters (e.g. `Vector{Float64}`). The notation `T where T <: SuperType` exists to denote sets of types.
- Arrays are deeply built into Julia
- Julia has many clever Array types (like `UnitRange`) to speed up array operations.